Topics: Data Modelling and Search Models
* Question anwering over tables

# Retrieve the data from the url

In [ ]:
import requests
import pandas as pd
import json

# Get table data
def get_table_data(url):

    response = requests.get(url)

    # store all tables in the tables list
    tables = []
    geometry_data = response.json()
    # loop through the dataset and convert tabular data to pandas dataframes
    for doc in geometry_data:
        table = pd.DataFrame(doc["data"], columns=doc["header"])
        tables.append(table)

    return tables, geometry_data

tables, geometry_data = get_table_data("https://raw.githubusercontent.com/aurioldegbelo/sis2024/refs/heads/main/vector_data/data_table.json")
print(len(tables))
tables[0]

2


,id,type,name,population,area,state_capital,geometry
0,s01,State,Nordrhein-Westfalen,18139116,34110.26,Düsseldorf,nrw_geom
1,s02,State,Hessen,6391360,21114.94,Wiesbaden,hessen_geom
2,s03,State,Rheinland-Pfalz,4159150,19854.21,Mainz,rlp_geom
3,s04,State,Niedersachsen,8140242,47709.82,Hannover,niedersachsen_geom


In [ ]:
tables[1]

,id,type,touches,url
0,nrw_geom,Geometry,"[hessen_geom, rlp_geom, niedersachsen_geom]",https://github.com/aurioldegbelo/sis2024/blob/...
1,hessen_geom,Geometry,"[nrw_geom, rlp_geom, niedersachsen_geom]",https://github.com/aurioldegbelo/sis2024/blob/...
2,rlp_geom,Geometry,"[nrw_geom, hessen_geom]",https://github.com/aurioldegbelo/sis2024/blob/...
3,niedersachsen_geom,Geometry,"[hessen_geom, nrw_geom]",https://github.com/aurioldegbelo/sis2024/blob/...


# Preprocess the table to include information about title and description

In [ ]:
# include the content of the table and its title + description
def preprocess_tables_with_metadata(tables: list, geometry_data):
    processed = []
    # loop through all tables
    for index, table in enumerate(tables):
        # convert the table to csv and
        processed_table = "\n".join([geometry_data[index]['title'], geometry_data[index]['description'], table.to_csv(index=False)])
        # add the processed table to processed list
        processed.append(processed_table)
    return processed

processed_table = preprocess_tables_with_metadata(tables, geometry_data)
print(processed_table[0])

list of states
This table contains all states in the database
id,type,name,population,area,state_capital,geometry
s01,State,Nordrhein-Westfalen,18139116,34110.26,Düsseldorf,nrw_geom
s02,State,Hessen,6391360,21114.94,Wiesbaden,hessen_geom
s03,State,Rheinland-Pfalz,4159150,19854.21,Mainz,rlp_geom
s04,State,Niedersachsen,8140242,47709.82,Hannover,niedersachsen_geom



In [ ]:
print(processed_table[1])

list of geometries
This table contains all geometries in the database their corresponding urls and the adjacent geometries
id,type,touches,url
nrw_geom,Geometry,"['hessen_geom', 'rlp_geom', 'niedersachsen_geom']",https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/nrw.geojson
hessen_geom,Geometry,"['nrw_geom', 'rlp_geom', 'niedersachsen_geom']",https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/hessen.geojson
rlp_geom,Geometry,"['nrw_geom', 'hessen_geom']",https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/rheinland-pfalz.geojson
niedersachsen_geom,Geometry,"['hessen_geom', 'nrw_geom']",https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/niedersachsen.geojson



# Create a retriever and load a model for question answering

In [ ]:
!pip install pinecone-client sentence_transformers

In [ ]:
import getpass
import os

if "PINECONE_API_KEY" not in os.environ:
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

Enter your Pinecone API key: ··········


In [ ]:
# initialize retriever
def initialize_retriever():
    import torch
    from sentence_transformers import SentenceTransformer

    # set device to GPU if available
    # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # load the table embedding model from huggingface models hub
    retriever = SentenceTransformer("deepset/all-mpnet-base-v2-table")

    return retriever


def create_pinecone_index():
    # initialize pinecone
    import os
    from pinecone import Pinecone, PodSpec

    pinecone_api_key = os.getenv("PINECONE_API_KEY")

    pc = Pinecone(api_key=pinecone_api_key)

    # create the pinecone index (see also https://docs.pinecone.io/guides/getting-started/quickstart)
    index_name = "table-qa-pc"

    # check if the table-qa index exists
    if index_name not in pc.list_indexes().names():
        # create the index if it does not exist
        pc.create_index(
            index_name,
            dimension=768,
            metric="cosine",
            spec=PodSpec(environment="gcp-starter")
            )

    # connect to table-qa index we created
    index = pc.Index(index_name)

    return index


def insert_embeddings_into_index (processed_tables, retriever, index):

    from tqdm.auto import tqdm
    # we will use batches of 64
    batch_size = 64

    for i in tqdm(range(0, len(processed_tables), batch_size)):
        # find end of batch
        i_end = min(i+batch_size, len(processed_tables))
        # extract batch
        batch = processed_tables[i:i_end]
        # generate embeddings for batch
        emb = retriever.encode(batch).tolist()
        # create unique IDs ranging from zero to the total number of tables in the dataset
        ids = [f"{idx}" for idx in range(i, i_end)]
        # add all to upsert list
        to_upsert = list(zip(ids, emb))
        # upsert/insert these records to pinecone
        index.upsert(vectors=to_upsert)

    return index

def load_tapas_model():

    from transformers import pipeline, TapasTokenizer, TapasForQuestionAnswering
    model_name = "google/tapas-base-finetuned-wtq"
    # load the tokenizer and the model from huggingface model hub
    tokenizer = TapasTokenizer.from_pretrained(model_name)
    model = TapasForQuestionAnswering.from_pretrained(model_name, local_files_only=False)
    # load the model and tokenizer into a question-answering pipeline
    pipe = pipeline(task ="table-question-answering",  model=model, tokenizer = tokenizer)
    return pipe

def load_tapex_model():
    # https://huggingface.co/microsoft/tapex-base?library=transformers
    from transformers import pipeline
    pipe = pipeline(task = "table-question-answering", model="microsoft/tapex-base")
    return pipe


def get_relevant_table(user_query, retriever, index, tables):
    # generate embedding for the query
    xq = retriever.encode([user_query]).tolist()
    # query pinecone index to find the table containing answer to the query
    result = index.query(vector=xq, top_k=1)
    # return the relevant table from the tables list
    print(result)
    return tables[int(result["matches"][0]["id"])]

def get_answer_from_table(table, user_query, pipe):
    # run the table and query through the question-answering pipeline
    # astype(str) because the the tapas tokenizer expects the data of the table to be text only, see https://huggingface.co/docs/transformers/model_doc/tapas
    answers = pipe(table=table.astype(str), query=user_query)
    return answers

# run all functions
def query_tables (url, pipe, user_query):
    tables, geometry_data = get_table_data(url)
    retriever = initialize_retriever()
    processed_tables = preprocess_tables_with_metadata(tables, geometry_data)
    initial_index = create_pinecone_index()
    final_index  = insert_embeddings_into_index(processed_tables, retriever, initial_index)
    final_index.describe_index_stats()
    relevant_table = get_relevant_table(user_query, retriever, final_index, tables)
    #print(relevant_table)
    answer = get_answer_from_table(relevant_table, user_query, pipe)
    #print(answer)
    return answer, relevant_table

In [ ]:
# load the model separately, to avoid loading it every time we have a new question
tapas_pipe = load_tapas_model() # TAPAS model
#tapex_pipe = load_tapex_model() # TAPEX model

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Play around with questions and answers

In [ ]:
data_url = "https://raw.githubusercontent.com/aurioldegbelo/sis2024/refs/heads/main/vector_data/data_table.json"

question1 = "How many states in the database?"
question2 = "How many geometries in the the database?"
question3 = "What is the population of Hessen?"
question4 = "What is the area of Hessen?"
question5 = "What is the capital of Hessen?"
question6 = "What is the geometry of Hessen?"
question7 = "What are the geometries of Hessen and Niedersachsen?"
question8 = "What is the url of the geometry of Hessen?"


answer, relevant_table = query_tables (data_url, tapas_pipe, question8)

print(answer)
#print(relevant_table)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

{'matches': [{'id': '1', 'score': 0.556921065, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}


/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:2673: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  text = normalize_for_match(row[col_index].text)
/usr/local/lib/python3.10/dist-packages/transformers/models/tapas/tokenization_tapas.py:1472: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cell = row[col_index]


{'answer': 'SUM > https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/nrw.geojson, https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/rheinland-pfalz.geojson', 'coordinates': [(0, 3), (2, 3)], 'cells': ['https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/nrw.geojson', 'https://github.com/aurioldegbelo/sis2024/blob/main/vector_data/rheinland-pfalz.geojson'], 'aggregator': 'SUM'}


Additional Links
* https://rocm.blogs.amd.com/artificial-intelligence/TaPas/README.html